In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.__version__

'2.9.1'

In [49]:
from datasets import load_dataset
data = load_dataset('conll2003')

Found cached dataset conll2003 (C:/Users/sajit/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 120.09it/s]


In [6]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [50]:
oov_token = '[UNK]'
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token, lower=True)
tokenizer.fit_on_texts([' '.join(x) for x in data['train']['tokens']])
len(tokenizer.word_index)


17680

In [51]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
  tokenized_inputs = [tokenizer.texts_to_sequences(token) for token in examples['tokens']]
  new_tokenized_inputs = []
  labels = []
  word_ids_list = []
  for i,tokenized_input in enumerate(tokenized_inputs):
    ner_tags = examples['ner_tags'][i]
    label_ids = []
    word_ids = []
    tokenized_sentence = []
    for j,tokenized_words in enumerate(tokenized_input):
      if tokenized_words:
        tokenized_sentence.extend(tokenized_words)
        word_ids.extend([j]*len(tokenized_words))
        label_ids.append(ner_tags[j]+1)
        for k in range(len(tokenized_words)-1):
          label_ids.append(ner_tags[j]+1 if label_all_tokens else 0)
    labels.append(label_ids)
    word_ids_list.append(word_ids)
    new_tokenized_inputs.append(tokenized_sentence)
  return {'input_ids': new_tokenized_inputs, 'word_ids': word_ids_list, 'labels': labels}


In [52]:
def input_and_label_pad_sequence(examples, maxlen=100):
  return { 'input_ids': tf.keras.preprocessing.sequence.pad_sequences(examples['input_ids'],maxlen = maxlen, padding='post', truncating='post'),
           'labels': tf.keras.preprocessing.sequence.pad_sequences(examples['labels'],maxlen = maxlen, padding='post', truncating='post' )}

In [53]:
def create_padding_mask(examples):
    seq = 1 - tf.cast(tf.math.equal(examples['input_ids'], 0), tf.int32)
    return {"attention_mask": seq}

In [54]:
maxlen = 100

data = data.map(tokenize_and_align_labels, batched=True).map(input_and_label_pad_sequence, batched=True).map(create_padding_mask, batched=True)

Loading cached processed dataset at C:\Users\sajit\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-c04bed9a42356567.arrow
Loading cached processed dataset at C:\Users\sajit\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-f0840bc6d022c9de.arrow
Loading cached processed dataset at C:\Users\sajit\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-5c59ad2a74d550d1.arrow
Loading cached processed dataset at C:\Users\sajit\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-2cedf137c4bc3781.arrow
Loading cached processed dataset at C:\Users\sajit\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-307a6bcf631f76a9.arrow
Loadi

In [94]:
dataset = tf.data.Dataset.from_tensor_slices((data['train']['input_ids'],data['train']['attention_mask'],data['train']['labels'])).map(lambda x,y,z: ((x,y),z)).shuffle(10000).batch(64)
test = tf.data.Dataset.from_tensor_slices((data['test']['input_ids'],data['test']['attention_mask'],data['test']['labels'])).map(lambda x,y,z: ((x,y),z))


ValueError: Attempt to convert a value ({'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0], 'input_ids': [946, 10365, 206, 633, 5, 3951, 216, 5739, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'word_ids': [0, 1, 2, 3, 4, 5, 6, 7], 'labels': [4, 1, 8, 1, 1, 1, 8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1,

In [56]:
for x in dataset.take(3):
    print(x)

(<tf.Tensor: shape=(64, 200), dtype=int32, numpy=
array([[ 3286,    14,  3287, ...,     0,     0,     0],
       [    4,  9291,   218, ...,     0,     0,     0],
       [   40,   107,   255, ...,     0,     0,     0],
       ...,
       [   27,    19,  1608, ...,     0,     0,     0],
       [   23,  3527, 11665, ...,     0,     0,     0],
       [ 1685,  3077,   144, ...,     0,     0,     0]])>, <tf.Tensor: shape=(64, 100), dtype=int32, numpy=
array([[4, 1, 4, ..., 0, 0, 0],
       [1, 6, 1, ..., 0, 0, 0],
       [1, 8, 9, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       [2, 3, 6, ..., 0, 0, 0]])>)
(<tf.Tensor: shape=(64, 200), dtype=int32, numpy=
array([[ 145,   19,  687, ...,    0,    0,    0],
       [   2,  602,   34, ...,    0,    0,    0],
       [2787,   20,   21, ...,    0,    0,    0],
       ...,
       [2631,  274, 1263, ...,    0,    0,    0],
       [ 610,   93,    0, ...,    0,    0,    0],
       [ 292, 3429,   30, ..., 

In [25]:
# @tf.keras.utils.register_keras_serializable()
def positional_encoding(positions, d_model):

    position = np.arange(positions)[:, np.newaxis]
    k = np.arange(d_model)[np.newaxis, :]
    i = k // 2

    # initialize a matrix angle_rads of all the angles
    angle_rates = 1 / np.power(10000, (2 * i) / np.float32(d_model))
    angle_rads = position * angle_rates

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [26]:
# @tf.keras.utils.register_keras_serializable()
def create_padding_mask(decoder_token_ids):

    seq = 1 - tf.cast(tf.math.equal(decoder_token_ids, 0), tf.float32)
    return seq[:, tf.newaxis, :]

In [27]:
# @tf.keras.utils.register_keras_serializable()
def FullyConnected(embedding_dim, fully_connected_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(fully_connected_dim, activation='relu'),
      tf.keras.layers.Dense(embedding_dim)
  ])

In [28]:
# @tf.keras.utils.register_keras_serializable()
class EncoderLayer(tf.keras.layers.Layer):

  def __init__(self, embedding_dim, num_heads, fully_connected_dim,
               dropout_rate=0.1, layernorm_eps=1e-6):

    super(EncoderLayer, self).__init__()

    self.mha = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=embedding_dim,
        dropout=dropout_rate
    )

    self.ffn = FullyConnected(
        embedding_dim=embedding_dim,
        fully_connected_dim=fully_connected_dim
    )

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=layernorm_eps)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=layernorm_eps)

    self.dropout_ffn = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, training, mask):

    self_mha_output = self.mha(x,x,x,mask)
    skip_x_attention = self.layernorm1(x+self_mha_output)
    ffn_output = self.ffn(skip_x_attention)
    ffn_output = self.dropout_ffn(ffn_output, training=training)
    encoder_layer_out = self.layernorm2(skip_x_attention+ffn_output)

    return encoder_layer_out

In [29]:
# @tf.keras.utils.register_keras_serializable()
class Encoder(tf.keras.layers.Layer):

  def __init__(self, num_layers, embedding_dim, num_heads, fully_connected_dim, vocab_size,
               maximum_position_encoding, dropout_rate=0.1, layernorm_eps=1e-6):
    super(Encoder, self).__init__()

    self.embedding_dim = embedding_dim

    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.embedding_dim)

    self.enc_layers = [EncoderLayer(embedding_dim=embedding_dim,
                                    num_heads=num_heads,
                                    fully_connected_dim=fully_connected_dim,
                                    dropout_rate=dropout_rate,
                                    layernorm_eps=layernorm_eps
                                    )
                      for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, training, mask):

     seq_len = tf.shape(x)[1]

     x = self.embedding(x)

     x*=tf.math.sqrt(tf.cast(self.embedding_dim, tf.float32))

     x+=self.pos_encoding[:,:seq_len,:]

     x = self.dropout(x, training=training)

     for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

     return x

In [74]:
# @tf.keras.utils.register_keras_serializable()
class NERModel(tf.keras.Model):

  def __init__(self, num_tags, num_layers, embedding_dim, num_heads, fully_connected_dim,
               vocab_size, max_positional_encoding,dropout_rate=0.1,
               layernorm_eps=1e-6):

    super(NERModel, self).__init__()

    self.encoder = Encoder(     num_layers=num_layers,
                                embedding_dim=embedding_dim,
                                num_heads=num_heads,
                                fully_connected_dim=fully_connected_dim,
                                vocab_size=vocab_size,
                                maximum_position_encoding=max_positional_encoding,
                                dropout_rate=dropout_rate,
                                layernorm_eps=layernorm_eps)
    self.create_padding_mask = create_padding_mask
    self.dropout1 = tf.keras.layers.Dropout(rate=dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(rate=dropout_rate)
    self.ffn = tf.keras.layers.Dense(fully_connected_dim, activation='relu')
    self.ffn_final = tf.keras.layers.Dense(num_tags,activation='softmax')

  def call(self, x, training):
    x, mask = x
    mask = mask[:,tf.newaxis,:]
    x = self.encoder(x,training, mask)
    x = self.dropout1(x, training = training)
    x = self.ffn(x)
    x = self.dropout2(x, training=training)
    x = self.ffn_final(x)
    return x




In [31]:
data['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [75]:
num_layers = 2
num_tags = 10
vocab_size = len(tokenizer.word_index)+1
embedding_dim = 100
fully_connected_dim = 100
num_heads = 3
positional_encoding_length = 150

transformer = NERModel(
    num_tags,
    num_layers,
    embedding_dim,
    num_heads,
    fully_connected_dim,
    vocab_size,
    positional_encoding_length,

)

In [35]:
class CustomNonPaddingTokenLoss(tf.keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

    def call(self, y_true, y_pred):
        mask = tf.math.logical_not(tf.math.equal(y_true, 0))
        loss_ = self.loss_object(y_true, y_pred)

        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask

        return tf.reduce_sum(loss_)/tf.reduce_sum(mask)




In [36]:
def acc(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    mask = tf.cast(mask,dtype=tf.float32)
    pred = tf.cast(tf.argmax(y_pred,axis=-1),dtype=tf.float32)
    correct = tf.cast(tf.equal(y_true,pred),dtype=tf.float32)
    correct*=mask
    return tf.reduce_sum(correct)/tf.reduce_sum(mask)


In [38]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = tf.cast(d_model, dtype=tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(embedding_dim)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [76]:
transformer.compile(optimizer=optimizer, loss=[CustomNonPaddingTokenLoss()], metrics = [acc])

In [78]:
transformer.fit(dataset,
                epochs = 1)

KeyboardInterrupt: 

In [38]:
transformer.fit(np.array(data['train']['input_ids']),np.array(data['train']['labels']),
                validation_data = (np.array(data['validation']['input_ids']),np.array(data['validation']['labels'])),
                epochs = 10)

Epoch 1/10
439/439 [==============================] - 10s 20ms/step - loss: 0.6173 - acc: 0.8375 - val_loss: 0.3979 - val_acc: 0.8832
Epoch 2/10
439/439 [==============================] - 9s 19ms/step - loss: 0.3000 - acc: 0.9122 - val_loss: 0.2944 - val_acc: 0.9089
Epoch 3/10
439/439 [==============================] - 8s 19ms/step - loss: 0.1908 - acc: 0.9437 - val_loss: 0.2673 - val_acc: 0.9198
Epoch 4/10
439/439 [==============================] - 8s 19ms/step - loss: 0.1336 - acc: 0.9600 - val_loss: 0.2809 - val_acc: 0.9200
Epoch 5/10
439/439 [==============================] - 8s 19ms/step - loss: 0.1037 - acc: 0.9681 - val_loss: 0.2557 - val_acc: 0.9260
Epoch 6/10
439/439 [==============================] - 8s 19ms/step - loss: 0.0842 - acc: 0.9738 - val_loss: 0.2724 - val_acc: 0.9215
Epoch 7/10
439/439 [==============================] - 8s 18ms/step - loss: 0.0722 - acc: 0.9772 - val_loss: 0.2609 - val_acc: 0.9355
Epoch 8/10
439/439 [==============================] - 8s 18ms/step -

In [80]:
example = data['test'][11]
example['tokens'],example['ner_tags'],example['word_ids'],example['labels'][:len(example['word_ids'])]

(['It',
  'was',
  'the',
  'second',
  'costly',
  'blunder',
  'by',
  'Syria',
  'in',
  'four',
  'minutes',
  '.'],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1])

In [93]:
transformer.predict(test.batch(64))

54/54 [==============================] - 1s 11ms/step


array([[[8.37833388e-04, 9.22988534e-01, 1.99593697e-02, ...,
         3.94196343e-03, 7.31733255e-03, 2.78275367e-03],
        [5.36015676e-03, 5.47678359e-02, 1.30722553e-01, ...,
         2.70101726e-02, 1.22728571e-01, 1.05833998e-02],
        [1.02137588e-03, 9.32893574e-01, 1.73388943e-02, ...,
         1.54023687e-03, 2.87428312e-03, 1.71657803e-03],
        ...,
        [4.85457107e-03, 2.46248335e-01, 4.60228622e-02, ...,
         2.92257909e-02, 3.42960320e-02, 1.92829035e-02],
        [4.28594509e-03, 2.46103957e-01, 5.73346429e-02, ...,
         4.14798558e-02, 4.62210290e-02, 2.35845707e-02],
        [3.46482103e-03, 2.04038277e-01, 6.56586364e-02, ...,
         4.79328707e-02, 5.56591153e-02, 2.11000573e-02]],

       [[1.29649590e-03, 5.94195127e-01, 5.91423027e-02, ...,
         9.45031643e-03, 3.33197340e-02, 9.30159818e-03],
        [1.22464180e-03, 6.68993115e-01, 4.87397276e-02, ...,
         7.43546244e-03, 2.60595009e-02, 6.53699739e-03],
        [2.66731111e-03, 

In [81]:
[np.argmax(x) for x in transformer(np.array([example['input_ids']]))[0]][:len(example['word_ids'])]

ValueError: Exception encountered when calling layer "ner_model_7" (type NERModel).

not enough values to unpack (expected 2, got 1)

Call arguments received by layer "ner_model_7" (type NERModel):
  • x=tf.Tensor(shape=(1, 100), dtype=int32)
  • training=None

In [39]:
transformer.save_weights('model')

In [40]:
dummy = NERModel(
    num_tags,
    num_layers,
    embedding_dim,
    num_heads,
    fully_connected_dim,
    vocab_size,
    positional_encoding_length,

)
dummy.load_weights('model')

In [28]:
tf.keras.models.load_model('model')

ValueError: Unable to create a Keras model from SavedModel at model. This SavedModel was exported with `tf.saved_model.save`, and lacks the Keras metadata file. Please save your Keras model by calling `model.save`or `tf.keras.models.save_model`. Note that you can still load this SavedModel with `tf.saved_model.load`.

In [41]:
[np.argmax(x) for x in dummy(np.array([example['input_ids']]))[0]][:len(example['word_ids'])]

[1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1]